In [ ]:
!nvidia-smi

Sun Nov 14 14:21:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip -q install iterative-stratification

In [ ]:
import os
import gc
import sys
import time
import random
import operator
import typing as tp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd,numpy as np
from copy import deepcopy as dp

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# Tabnet 
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/AV/AmExpert2021/input/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/AV/AmExpert2021/input/test.csv")
ss_df = pd.read_csv("/content/drive/MyDrive/AV/AmExpert2021/input/sample_submission.csv")

train_df.shape, test_df.shape, ss_df.shape

((37748, 9), (20327, 8), (20327, 2))

In [ ]:
# train_df = train_df[~train_df.drop(columns='Customer_ID').duplicated()].reset_index(drop=True)
# train_df.shape

In [ ]:
train_df['PHB1_len'] = train_df['Product_Holding_B1'].apply(lambda x: len(eval(x)))
test_df['PHB1_len'] = test_df['Product_Holding_B1'].apply(lambda x: len(eval(x)))

In [ ]:
cat_cols = ['Gender', 'City_Category', 'Customer_Category']

for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])

In [ ]:
CCtrain_df = pd.get_dummies(train_df['Customer_Category'], prefix='Customer_Category')
CCtest_df = pd.get_dummies(test_df['Customer_Category'], prefix='Customer_Category')

train_df[CCtrain_df.columns] = CCtrain_df
test_df[CCtest_df.columns] = CCtest_df

In [ ]:
num_cols = ['Age', 'Vintage', 'PHB1_len']

for col in num_cols:
    mms = MinMaxScaler()
    train_df[col] = mms.fit_transform(train_df[col].values.reshape(-1, 1))
    test_df[col] = mms.transform(test_df[col].values.reshape(-1, 1))

In [ ]:
PHB1_list = train_df.Product_Holding_B1.apply(eval).values.tolist()
PHB2_list = train_df.Product_Holding_B2.apply(eval).values.tolist()
tPHB1_list = test_df.Product_Holding_B1.apply(eval).values.tolist()

In [ ]:
mlb1 = MultiLabelBinarizer()
mlb2 = MultiLabelBinarizer()

PHB1_onehot = mlb1.fit_transform(PHB1_list)
PHB2_onehot = mlb2.fit_transform(PHB2_list)
tPHB1_onehot = mlb1.transform(tPHB1_list)

PHB1_onehot.shape, PHB2_onehot.shape, tPHB1_onehot.shape

((37748, 22), (37748, 20), (20327, 22))

In [ ]:
train_df[mlb1.classes_] = PHB1_onehot
test_df[mlb1.classes_] = tPHB1_onehot

In [ ]:
cid_list = []
target_list = []
for cid, ph_list in zip(train_df.Customer_ID.values, PHB2_list):
    for l in ph_list:
        cid_list.append(cid)
        target_list.append(l)

target_df = pd.DataFrame({
    "Customer_ID" : cid_list,
    "Target" : target_list
})

print(len(cid_list), len(target_list))

target_df.shape

52057 52057


(52057, 2)

In [ ]:
def pca_pre(tr,te, n_comp, feat_raw, feat_new):
    pca = PCA(n_components=n_comp, random_state=42)
    tr2 = pd.DataFrame(pca.fit_transform(tr[feat_raw]),columns=feat_new)
    #va2 = pd.DataFrame(pca.transform(va[feat_raw]),columns=feat_new)
    te2 = pd.DataFrame(pca.transform(te[feat_raw]),columns=feat_new)
    return(tr2,te2)

n_comp1 = 5
feat_cols = mlb1.classes_
pca_feat_g = [f'pca-{i}' for i in range(n_comp1)]

x_tr_g_pca,x_te_g_pca = pca_pre(train_df, test_df,
                                            n_comp1, feat_cols ,pca_feat_g)
train_df = pd.concat([train_df, x_tr_g_pca],axis = 1)
# x_valid = pd.concat([x_valid,x_va_g_pca],axis = 1)
test_df  = pd.concat([test_df, x_te_g_pca],axis = 1)

train_df.shape, test_df.shape

((37748, 40), (20327, 39))

In [ ]:
train_df = target_df.merge(train_df, how='left')

In [ ]:
target_le = LabelEncoder()
train_df['Target'] = target_le.fit_transform(train_df['Target'])

In [ ]:
drop_cols = ['Customer_ID', 'Customer_Category', 'Target', 'Product_Holding_B1', 'Product_Holding_B2']
train_cols = train_df.drop(columns=drop_cols).columns.values
train_cols

array(['Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category',
       'PHB1_len', 'Customer_Category_0', 'Customer_Category_1',
       'Customer_Category_2', 'P00', 'P1', 'P10', 'P11', 'P12', 'P13',
       'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P2', 'P20', 'P21', 'P3',
       'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'pca-0', 'pca-1', 'pca-2',
       'pca-3', 'pca-4'], dtype=object)

In [ ]:
target = train_df['Target'].values
X_test = test_df[train_cols].values

In [ ]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

# num_features=len(feature_cols)
# num_targets=len(target_cols)
# num_targets_0=len(target_nonsc_cols2)
hidden_size=4096

In [ ]:

def g_table(list1):
    table_dic = {}
    for i in list1:
        if i not in table_dic.keys():
            table_dic[i] = 1
        else:
            table_dic[i] += 1
    return(table_dic)

tar_freq = np.array([np.min(list(g_table(PHB2_onehot[:, i]))) for i in range(PHB2_onehot.shape[1])])
tar_weight0 = np.array([np.log(i+100) for i in tar_freq])
tar_weight0_min = dp(np.min(tar_weight0))
tar_weight = tar_weight0_min/tar_weight0
pos_weight = torch.tensor(tar_weight).to(DEVICE)

wgt_bce = dp(F.binary_cross_entropy_with_logits)
wgt_bce.__defaults__ = (None, None, None, 'mean', pos_weight)

from torch.nn.modules.loss import _WeightedLoss
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight,
                                                    pos_weight = pos_weight)
        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
class TrainDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return (self.features.shape[0])

    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx])            
        }
        return dct

class TestDataset:
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return (self.features.shape[0])

    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0

    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        final_loss += loss.item()

    final_loss /= len(dataloader)

    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []

    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())

    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)

    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []

    for data in dataloader:
        inputs = data['x'].to(device)
        with torch.no_grad():
            outputs = model(inputs)

        preds.append(outputs.sigmoid().detach().cpu().numpy())

    preds = np.concatenate(preds)

    return preds


In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed = 1001

seed_everything(seed)

In [ ]:
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold


In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        cha_1 = 256
        cha_2 = 512
        cha_3 = 512

        cha_1_reshape = int(hidden_size/cha_1)
        cha_po_1 = int(hidden_size/cha_1/2)
        cha_po_2 = int(hidden_size/cha_1/2/2) * cha_3

        self.cha_1 = cha_1
        self.cha_2 = cha_2
        self.cha_3 = cha_3
        self.cha_1_reshape = cha_1_reshape
        self.cha_po_1 = cha_po_1
        self.cha_po_2 = cha_po_2

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

        self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
        self.dropout_c1 = nn.Dropout(0.1)
        self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1,cha_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = cha_po_1)

        self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2 = nn.Dropout(0.1)
        self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

        self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_1 = nn.Dropout(0.3)
        self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

        self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_2 = nn.Dropout(0.2)
        self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_3, kernel_size = 5, stride = 1, padding=2, bias=True),dim=None)

        self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

    def forward(self, x):

        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.celu(self.dense1(x), alpha=0.06)

        x = x.reshape(x.shape[0],self.cha_1,
                        self.cha_1_reshape)

        x = self.batch_norm_c1(x)
        x = self.dropout_c1(x)
        x = F.relu(self.conv1(x))

        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = F.relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c2_1(x)
        x = self.dropout_c2_1(x)
        x = F.relu(self.conv2_1(x))

        x = self.batch_norm_c2_2(x)
        x = self.dropout_c2_2(x)
        x = F.relu(self.conv2_2(x))
        x =  x * x_s

        x = self.max_po_c2(x)

        x = self.flt(x)

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)

        return x

In [ ]:
hidden_size=4096

In [ ]:
#skf = MultilabelStratifiedKFold(n_splits= 5, shuffle=True, random_state=seed)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

test_predictions = []
train_pred = []
train_true = []

for idx, (train_idx, valid_idx) in enumerate(skf.split(train_df, target)):
    fold = idx

    X_train = train_df.iloc[train_idx][train_cols].values
    #y_train = PHB2_onehot[train_idx]
    y_train = target[train_idx]

    X_valid = train_df.iloc[valid_idx][train_cols].values
    #y_valid = PHB2_onehot[valid_idx]
    y_valid = target[valid_idx]
    #valid_true = np.array(PHB2_list)[valid_idx]

    num_features = X_train.shape[1]
    num_targets = 20 #y_train.shape[1]

    print("Trian :", X_train.shape, y_train.shape)
    print("Valid :", X_valid.shape, y_valid.shape)

    train_dataset = TrainDataset(X_train, y_train)
    valid_dataset = TrainDataset(X_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )

    model.to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                                max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))

    loss_tr = nn.CrossEntropyLoss() #nn.MultiLabelSoftMarginLoss() #nn.BCEWithLogitsLoss(,) #SmoothBCEwLogits(smoothing = 0.001)
    loss_va = nn.CrossEntropyLoss() #nn.BCEWithLogitsLoss()    

    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0

    #oof = np.zeros((len(train), len(target_cols)))
    best_loss = np.inf
    best_score = -np.inf

    mod_name = f"FOLD_mod11_{seed}_{fold}_.pth"
        
    for epoch in range(EPOCHS):

        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, loss_va, validloader, DEVICE)

        # valid_pred = np.argsort(valid_preds)[:, -3:][:, ::-1]
        # final_test = []
        # for i in valid_pred:
        #     final_test.append([mlb2.classes_[int(ii)] for ii in i])

        # score = mapk(valid_true , final_test)

        preds = np.argsort(valid_preds)[:, -3:][:, ::-1]
        label = y_valid.reshape(len(y_valid), 1)

        score =  mapk(label, preds, k=3)

        print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch},train_loss: {train_loss}, valid_loss: {valid_loss}, score: {score}")

        if valid_loss < best_loss:

            best_loss = valid_loss
            #oof[val_idx] = valid_preds
            torch.save(model.state_dict(), mod_name)
        
        # if score > best_score:
        #     best_score = score
        #     #oof[val_idx] = valid_preds
        #     best_preds = valid_preds
        #     torch.save(model.state_dict(), mod_name)

        elif(EARLY_STOP == True):

            early_step += 1
            if (early_step >= early_stopping_steps):
                break
        
    train_pred.append(best_preds)
    train_true.append(y_valid)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Trian : (41645, 36) (41645,)
Valid : (10412, 36) (10412,)
SEED: 1001, FOLD: 0, EPOCH: 0,train_loss: 2.0611280484433556, valid_loss: 2.2237584649062736, score: 0.4510980919451914
SEED: 1001, FOLD: 0, EPOCH: 1,train_loss: 1.6897328068142288, valid_loss: 1.598298308325977, score: 0.4863138686131387
SEED: 1001, FOLD: 0, EPOCH: 2,train_loss: 1.6278273316980139, valid_loss: 1.687226075951646, score: 0.49667050838775767
SEED: 1001, FOLD: 0, EPOCH: 3,train_loss: 1.5985407116222967, valid_loss: 1.5769040540951054, score: 0.4887629658086823
SEED: 1001, FOLD: 0, EPOCH: 4,train_loss: 1.5875833608621468, valid_loss: 1.5511468052864075, score: 0.4962863362786529
SEED: 1001, FOLD: 0, EPOCH: 5,train_loss: 1.5800578308983084, valid_loss: 1.553842752445035, score: 0.4974708669483928
SEED: 1001, FOLD: 0, EPOCH: 6,train_loss: 1.5743636514511576, valid_loss: 1.5350382764164994, score: 0.49750288129081827
SEED: 1001, FOLD: 0, EPOCH: 7,train_loss: 1.5704322391492458, valid_loss: 1.567258746158786, score: 0.4

In [ ]:
import glob

In [ ]:
models_path = glob.glob("*.pth")
models_path

['FOLD_mod11_1001_1_.pth',
 'FOLD_mod11_1001_4_.pth',
 'FOLD_mod11_1001_2_.pth',
 'FOLD_mod11_1001_0_.pth',
 'FOLD_mod11_1001_3_.pth']

In [ ]:
test_predictions = []

for m_path in models_path:
    testdataset = TestDataset(X_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)

    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )

    model.load_state_dict(torch.load(mod_name))
    model.to(DEVICE)

    predictions = inference_fn(model, testloader, DEVICE)
    test_predictions.append(predictions)


In [ ]:
final_predictions_mean = np.array(test_predictions).mean(axis=0)
final_predictions_mean.shape

(20327, 20)

In [ ]:
np.save("/content/drive/MyDrive/AV/AmExpert2021/input/CNN_D_M_C_pre.npy", final_predictions_mean)

In [ ]:
final_predictions = np.argsort(final_predictions_mean)[:, -3:][:, ::-1]

In [ ]:
final_test = []
for i in final_predictions:
    final_test.append([mlb2.classes_[int(ii)] for ii in i])

In [ ]:
ss_df['Product_Holding_B2'] = final_test
ss_df.to_csv("/content/drive/MyDrive/AV/AmExpert2021/input/CNN_D_M_C.csv",index=False)

In [ ]:
ss_df

,Customer_ID,Product_Holding_B2
0,CC372708,"[P8, P10, P4]"
1,CC216072,"[P8, P1, P10]"
2,CC387629,"[P16, P13, P00]"
3,CC389228,"[P8, P12, P9]"
4,CC394445,"[P00, P8, P1]"
...,...,...
20322,CC303542,"[P8, P6, P00]"
20323,CC266713,"[P8, P12, P6]"
20324,CC393639,"[P8, P9, P6]"
20325,CC285013,"[P1, P6, P7]"
